### [00] 실습을 시작하기 전에

In [ ]:
pip install newspaper3k

### [01] newspaper3k 라이브러리 사용해보기

In [ ]:
from newspaper import Article

# ➊ 파싱할 뉴스 기사 URL 지정
url = 'https://news.nate.com/view/20230211n06158?mid=n0601'

# ➋ 언어를 한국어로 설정하고 URL을 전달해 Article 클래스의 객체 생성
article = Article(url, language='ko')

# ➌ 지정된 웹페이지를 다운로드
article.download()

# ➍ 다운로드한 웹페이지를 분석하고 필요한 정보를 추출
article.parse()

In [ ]:
# 기사 제목 출력
print('기사 제목 :',article.title)
print('')

# 기사 내용 출력
print('기사 내용 :')
print(article.text)

### [05] 뉴스 기사 크롤러 만들기


In [ ]:
page_num = 2 # ➊ 페이지 번호
category1 = 'its99' # ➋ IT/과학 일반 카테고리 코드1
category2 = '0607' # ➋ IT/과학 일반 카테고리 코드2
date = 20240912 # ➌ 날짜

# ➍ 변수를 넣어 URL 생성
url = 'https://news.nate.com/subsection?cate='+category1+'&mid=n'+category2+'&type=c&date='+str(date)+'?page='+str(page_num)
print(url)

In [ ]:
import requests

news = requests.get(url)
news.text

In [ ]:
import requests
from bs4 import BeautifulSoup


# ➊ 페이지 수, 카테고리, 날짜를 전달받아 뉴스 기사의 URL을 추출하는 함수 make_urllist( ) 정의
def make_urllist(page_num, category1, category2, date):
  urllist= []
    
  # ➋ 1페이지부터 page_num에 정해진 페이지만큼 for문으로 작업을 반복
  for i in range(1, page_num + 1):
    # ➌ 함수 호출 시 전달받은 인자를 조합해 다음 뉴스 페이지의 URL 생성
    url = 'https://news.nate.com/subsection?cate='+category1+'&mid=n'+category2+'&type=c&date='+str(date)+'?page='+str(i)
    # ➍ requets.get()으로 웹페이지의 HTML 문서 추출
    news = requests.get(url)
    # ➎ BeautifulSoup()으로 가져온 HTML 문서 파싱
    soup = BeautifulSoup(news.text, 'html.parser')
    # ➏ 뉴스 목록에 포함된 각 뉴스의 제목에 해당하는 a 태그를 가져와 리스트로 저장
    news_list = soup.select('.postSubjectContent .mduSubjectList .mlt01 a')
      
    # ➐ a의 href 속성값(url)만 추출해 urllist 리스트에 저장
    for line in news_list:
      urllist.append(line.get('href'))
        
  # ➑ 함수 실행의 결과로 urllist 리스트 반환
  return urllist

In [ ]:
url_list = make_urllist(2, 'its99', '0607', 20240912)
print('뉴스 기사의 개수 :', len(url_list))

In [ ]:
print(url_list[:5])

In [ ]:
from newspaper import Article
import pandas as pd


# IT/과학 카테고리의 하위 카테고리 키워드와 실제 카테고리명을 매핑하는 딕셔너리 생성
idx2word = {'0602' : '과학', '0604' : '컴퓨터/디지털', '0605' : '뉴미디어/통신', '0607' : 'IT/과학 일반'}

# ➊ 뉴스 기사 URL 리스트와 카테고리 키워드를 받아 기사 본문을 추출하는 함수 make_data() 생성
def make_data(urllist, category1, category2):
  text_list = []


  # ➋ 주어진 URL 리스트를 반복하면서 각 뉴스 기사를 처리
  for url in urllist:
    url = 'https:' + url
    article = Article(url, language='ko') # Article 객체 생성, 언어는 한국어로 설정
    article.download() # 기사를 다운로드
    article.parse() # 기사를 파싱
    text_list.append(article.text) # 파싱된 텍스트를 리스트에 추가
  # ➌ 'news' 열의 행마다 각 뉴스 기사 본문이 저장된 데이터프레임 생성
  df = pd.DataFrame({'news': text_list})
  # ➍ 기사 본문에서 줄바꿈 문자를 공백으로 치환
  df['news'] = df['news'].str.replace('\n',' ') # 가져온 뉴스 내용에서 \n을 공백으로 바꿔주는 코드
  # ➎ 데이터프레임에 category 열을 추가하고 딕셔너리 idx2word에서 해당 keyword에 대응하는 값 할당
  df['category'] = idx2word[category2]
  # ➏ 완성된 데이터프레임 반환
  return df


# ➐ make_data 함수 호출, 특정 키워드에 해당하는 URL 리스트와 함께
data = make_data(url_list, 'its99', '0607')
data[:10] # 상위 10개 출력

In [ ]:
# url_list의 첫 번째 요소를 출력
print('https:' + url_list[0])

In [ ]:
# 카테고리 코드 리스트 생성
category_list = [('its01','0602'), ('its03','0604'), ('its04','0605'), ('its99','0607')]


# ➊ 페이지 수와 키워드 리스트, 날짜를 인자로 받아 뉴스 기사를 추출하는 make_total_data()함수 생성
def make_total_data(page_num, keyword_list, date):
  # ➋ 초기 데이터프레임을 None으로 설정
  df = None
  # ➌ keyword_list의 요소를 순회하며 다음의 작업을 반복
  for cate1, cate2 in category_list:
    # ➍ page_num과 date, 그리고 현재 카테고리 코드(cate1, cate2)를 전달해 make_urllist()실행
    url_list = make_urllist(page_num, cate1, cate2, date)
    # ➎ url_list와 현재 카테고리 코드(cate1, cate2)를 전달해 make_data() 실행
    df_temp = make_data(url_list, cate1, cate2)
    # ➏ 진행 상황을 확인하기 위한 문자열 출력
    print(f'{cate1} 카테고리의 뉴스 기사 데이터 추출을 완료했습니다.')
    # ➐ 데이터프레임 df에 값이 존재하면 df와 df_temp를 병합
    if df is not None:
      df = pd.concat([df, df_temp], ignore_index=True)
    # ➑ df가 None이면 df_temp를 df로 할당
    else:
      df = df_temp


  # ➒ 모든 카테고리에 대한 기사 수집이 완료되면 최종 df를 반환
  return df

In [ ]:
# 24년 9월 12일의 각 카테고리별 뉴스 기사를 2페이지까지 수집해 데이터프레임에 저장
df = make_total_data(2, category_list, 20240912)

In [ ]:
print('뉴스 기사의 개수 :', len(df))

In [ ]:
# sample( ) 함수로 데이터프레임의 데이터를 10개만 랜덤으로 뽑아서 출력
df.sample(10)